<a href="https://colab.research.google.com/github/psmouli14/final_project/blob/main/few_shot_learning_with_omniglot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
from PIL import Image

import numpy as np
import tensorflow as tf

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving omniglot.zip to omniglot.zip


In [ ]:
!unzip omniglot.zip

Streaming output truncated to the last 5000 lines.
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_05.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_11.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_10.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_04.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_12.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_06.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_07.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_13.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_17.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_03.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_02.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_16.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_14.png  


In [ ]:
root_dir = 'omniglot/'
train_split_path = os.path.join(root_dir, 'splits', 'vinyals', 'train.txt')
print(train_split_path)

with open(train_split_path, 'r') as train_split:
    train_classes = [line.rstrip() for line in train_split.readlines()]
print(train_classes[2])

val_split_path = os.path.join(root_dir, 'splits', 'vinyals', 'val.txt')
print(val_split_path)

with open(val_split_path, 'r') as val_split:
    val_classes = [line.rstrip() for line in val_split.readlines()]
print(val_classes[2])

test_split_path = os.path.join(root_dir, 'splits', 'vinyals', 'test.txt')
print(test_split_path)

with open(test_split_path, 'r') as test_split:
    test_classes = [line.rstrip() for line in test_split.readlines()]
print(test_classes[2])

omniglot/splits/vinyals/train.txt
Angelic/character01/rot180
omniglot/splits/vinyals/val.txt
Hebrew/character01/rot180
omniglot/splits/vinyals/test.txt
Gurmukhi/character42/rot180


In [ ]:
no_train_classes = len(train_classes)
print(no_train_classes)

no_val_classes = len(val_classes)
print(no_val_classes)

no_test_classes = len(test_classes)
print(no_test_classes)

4112
688
1692


In [ ]:
#number of examples per class
num_examples = 20

#image width
img_width = 32

#image height
img_height = 32

#channels
channels = 1

In [ ]:
train_dataset = np.zeros([no_train_classes, num_examples, img_height, img_width], dtype=np.float32)
print(train_dataset.shape)

val_dataset = np.zeros([no_val_classes, num_examples, img_height, img_width], dtype=np.float32)
print(val_dataset.shape)

test_dataset = np.zeros([no_test_classes, num_examples, img_height, img_width], dtype=np.float32)
print(test_dataset.shape)


(4112, 20, 32, 32)
(688, 20, 32, 32)
(1692, 20, 32, 32)


In [ ]:
def populate_dataset(classes, dataset):
  for label, name in enumerate(classes):
    alphabet, character, rotation = name.split('/')
    rotation = float(rotation[3:])
    img_dir = os.path.join(root_dir, 'data', alphabet, character)
    img_files = sorted(glob.glob(os.path.join(img_dir, '*.png')))
    
    for index, img_file in enumerate(img_files):
      
      img = Image.open(img_file).resize((img_height, img_width)).rotate(rotation)
      img = np.asarray(img)
      img = 1 - img
      dataset[label, index] = img

  return dataset


train_dataset = populate_dataset(train_classes, train_dataset)
val_dataset = populate_dataset(val_classes, val_dataset)
test_dataset = populate_dataset(test_classes, test_dataset)


In [ ]:
#Increase the axis by 1 to accomodate channel
train_dataset = np.expand_dims(train_dataset, axis=-1)
val_dataset = np.expand_dims(val_dataset, axis=-1)
test_dataset = np.expand_dims(test_dataset, axis=-1)

print(train_dataset.shape)
print(val_dataset.shape)
print(test_dataset.shape)

(4112, 20, 32, 32, 1)
(688, 20, 32, 32, 1)
(1692, 20, 32, 32, 1)


In [10]:
# This will be used to get the next set of support and query dataset for the episode
def get_next_episode(dataset, num_way, num_shot, num_query, no_of_classes):
  support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
  query = np.zeros([num_way, num_query, img_height, img_width, channels], dtype=np.float32)
  episodic_classes = np.random.permutation(no_of_classes)[:num_way]

  for index, class_ in enumerate(episodic_classes):
    selected = np.random.permutation(num_examples)[:num_shot + num_query]
    
    support[index] = dataset[class_][selected[:num_shot]]
    query[index] = dataset[class_][selected[num_shot:]]
    
  return support, query

def euclidean_distance(a, b):

    N, D = tf.shape(a)[0], tf.shape(a)[1]
    M = tf.shape(b)[0]
    a = tf.tile(tf.expand_dims(a, axis=1), (1, M, 1))
    b = tf.tile(tf.expand_dims(b, axis=0), (N, 1, 1))
    return tf.reduce_mean(tf.square(a - b), axis=2)


In [11]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, GlobalMaxPooling2D
from tensorflow.keras import Model


class Prototypical(Model):
    
    def __init__(self, n_support, n_query, w, h, c):
       
        super(Prototypical, self).__init__()
        self.w, self.h, self.c = w, h, c

        # Encoder with 4 blocks
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2, 2)),

            tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2, 2)),

            tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2, 2)),

            tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2, 2)), Flatten()]
        )

    def call(self, support, query):
        n_class = support.shape[0]
        n_support = support.shape[1]
        n_query = query.shape[1]
        y = np.tile(np.arange(n_class)[:, np.newaxis], (1, n_query))
        y_onehot = tf.cast(tf.one_hot(y, n_class), tf.float32)

        # correct indices of support samples (just natural order)
        target_inds = tf.reshape(tf.range(n_class), [n_class, 1])
        target_inds = tf.tile(target_inds, [1, n_query])

        # merge support and query to forward through encoder
        cat = tf.concat([
            tf.reshape(support, [n_class * n_support,
                                 self.w, self.h, self.c]),
            tf.reshape(query, [n_class * n_query,
                               self.w, self.h, self.c])], axis=0)
        z = self.encoder(cat)

        # Divide embedding into support and query
        z_prototypes = tf.reshape(z[:n_class * n_support],
                                  [n_class, n_support, z.shape[-1]])
        
        z_prototypes = tf.math.reduce_mean(z_prototypes, axis=1)
        z_query = z[n_class * n_support:]

        # Calculate distances between query and prototypes
        dists = euclidean_distance(z_query, z_prototypes)

        #use softmax
        log_p_y = tf.nn.log_softmax(-dists, axis=-1)
        log_p_y = tf.reshape(log_p_y, [n_class, n_query, -1])
        
        loss = -tf.reduce_mean(tf.reshape(tf.reduce_sum(tf.multiply(y_onehot, log_p_y), axis=-1), [-1]))
        eq = tf.cast(tf.equal(
            tf.cast(tf.argmax(log_p_y, axis=-1), tf.int32), 
            tf.cast(y, tf.int32)), tf.float32)
        acc = tf.reduce_mean(eq)
        return loss, acc

    def save(self, model_path):
       
        self.encoder.save(model_path)

    def load(self, model_path):
       
        self.encoder(tf.zeros([1, self.w, self.h, self.c]))
        self.encoder.load_weights(model_path)

In [12]:
# Lists to hold values for N-way k-shots experiments
train_num_ways = [60, 60, 40, 40]
test_num_ways = [5, 5, 20, 20]
num_shots = [5, 1, 5, 1]
learning_rate = 0.001

In [13]:
#Run prototypical model with only training set
num_epochs = 20
num_episodes = 100
save_path = "./results/models/omniglot_train0.h5"

train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')


#number of classes
num_way = train_num_ways[0] 

#number of examples per class for support set
num_shot = num_shots[0]  

#number of query points
num_query = num_shots[0] 

least_loss = {'least_loss': 100.00}

support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

least_loss = {'least_loss': 100.00}

@tf.function
def train_step(support, query):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  train_acc.reset_states()

  for episode in range(num_episodes):
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    train_step(train_support, train_query)

  cur_loss = train_loss.result().numpy()
  if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100))

Saving new best model with loss:  1.8685445
Epoch 1, Loss: 1.8685444593429565, Accuracy: 58.74000930786133


Saving new best model with loss:  0.65386987
Epoch 2, Loss: 0.6538698673248291, Accuracy: 82.09666442871094


Saving new best model with loss:  0.43051377
Epoch 3, Loss: 0.43051376938819885, Accuracy: 87.80667114257812


Saving new best model with loss:  0.3056561
Epoch 4, Loss: 0.3056561052799225, Accuracy: 91.50334930419922


Saving new best model with loss:  0.23649561
Epoch 5, Loss: 0.23649561405181885, Accuracy: 93.30331420898438


...

Saving new best model with loss:  0.11247861
Epoch 15, Loss: 0.11247860640287399, Accuracy: 96.71334075927734


Saving new best model with loss:  0.10422583
Epoch 16, Loss: 0.10422582924365997, Accuracy: 96.91667938232422


Saving new best model with loss:  0.09971292
Epoch 17, Loss: 0.09971292316913605, Accuracy: 97.09000396728516


Saving new best model with loss:  0.09016416
Epoch 18, Loss: 0.09016416221857071, Accuracy: 97.32998657226562
Epoch 19, Loss: 0.09904658049345016, Accuracy: 97.10334014892578


Saving new best model with loss:  0.08822292
Epoch 20, Loss: 0.0882229208946228, Accuracy: 97.35335540771484


In [14]:
#Run prototypical model with only training set
# This will be using the second values from the lists for the experiment
num_epochs = 20
num_episodes = 100
save_path = "./results/models/omniglot_train1.h5"

train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')


#number of classes
num_way = train_num_ways[1] 

#number of examples per class for support set
num_shot = num_shots[1]  

#number of query points
num_query = num_shots[1] 

least_loss = {'least_loss': 100.00}

support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

least_loss = {'least_loss': 100.00}

@tf.function
def train_step(support, query):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  train_acc.reset_states()

  for episode in range(num_episodes):
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    train_step(train_support, train_query)

  cur_loss = train_loss.result().numpy()
  if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100))

Saving new best model with loss:  2.7890153
Epoch 1, Loss: 2.789015293121338, Accuracy: 36.0333366394043


Saving new best model with loss:  1.7985147
Epoch 2, Loss: 1.798514723777771, Accuracy: 55.3166389465332


Saving new best model with loss:  1.2796829
Epoch 3, Loss: 1.2796828746795654, Accuracy: 66.00001525878906


Saving new best model with loss:  0.966193
Epoch 4, Loss: 0.9661930203437805, Accuracy: 73.69999694824219


Saving new best model with loss:  0.78253835
Epoch 5, Loss: 0.7825383543968201, Accuracy: 78.5999984741211


...

Saving new best model with loss:  0.4784554
Epoch 12, Loss: 0.47845539450645447, Accuracy: 86.58333587646484
Epoch 13, Loss: 0.5023582577705383, Accuracy: 85.68331909179688
Epoch 14, Loss: 0.49165549874305725, Accuracy: 85.71665954589844


Saving new best model with loss:  0.46893302
Epoch 15, Loss: 0.46893301606178284, Accuracy: 87.28334045410156


Saving new best model with loss:  0.42954728
Epoch 16, Loss: 0.4295472800731659, Accuracy: 87.88334655761719
Epoch 17, Loss: 0.4504046142101288, Accuracy: 87.54998016357422
Epoch 18, Loss: 0.43568798899650574, Accuracy: 87.99996185302734


Saving new best model with loss:  0.41791317
Epoch 19, Loss: 0.41791316866874695, Accuracy: 88.23331451416016


Saving new best model with loss:  0.39165604
Epoch 20, Loss: 0.3916560411453247, Accuracy: 88.64999389648438


In [15]:
#Run prototypical model with only training set
# This will be using the third values from the lists for the experiment
num_epochs = 20
num_episodes = 100
save_path = "./results/models/omniglot_train2.h5"

train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')


#number of classes
num_way = train_num_ways[2] 

#number of examples per class for support set
num_shot = num_shots[2]  

#number of query points
num_query = num_shots[2] 

least_loss = {'least_loss': 100.00}

support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

least_loss = {'least_loss': 100.00}

@tf.function
def train_step(support, query):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  train_acc.reset_states()

  for episode in range(num_episodes):
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    train_step(train_support, train_query)

  cur_loss = train_loss.result().numpy()
  if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100))

Saving new best model with loss:  1.5922908
Epoch 1, Loss: 1.5922907590866089, Accuracy: 63.304988861083984


Saving new best model with loss:  0.6238901
Epoch 2, Loss: 0.6238901019096375, Accuracy: 82.70997619628906


Saving new best model with loss:  0.41510868
Epoch 3, Loss: 0.41510868072509766, Accuracy: 88.46000671386719


Saving new best model with loss:  0.30040586
Epoch 4, Loss: 0.3004058599472046, Accuracy: 91.15496826171875


Saving new best model with loss:  0.21886522
Epoch 5, Loss: 0.21886521577835083, Accuracy: 93.60498809814453


...

Saving new best model with loss:  0.12188352
Epoch 10, Loss: 0.12188351899385452, Accuracy: 96.3949966430664
Epoch 11, Loss: 0.12225532531738281, Accuracy: 96.47498321533203
Epoch 12, Loss: 0.12399077415466309, Accuracy: 96.52001953125


Saving new best model with loss:  0.11435556
Epoch 13, Loss: 0.11435555666685104, Accuracy: 96.47500610351562


Saving new best model with loss:  0.10260033
Epoch 14, Loss: 0.1026003286242485, Accuracy: 96.9699935913086
Epoch 15, Loss: 0.1174960732460022, Accuracy: 96.80000305175781
Epoch 16, Loss: 0.10826302319765091, Accuracy: 96.85002136230469


Saving new best model with loss:  0.09518498
Epoch 17, Loss: 0.09518498182296753, Accuracy: 97.20000457763672


Saving new best model with loss:  0.08883213
Epoch 18, Loss: 0.08883213251829147, Accuracy: 97.3800048828125
Epoch 19, Loss: 0.09964302182197571, Accuracy: 97.15999603271484
Epoch 20, Loss: 0.0959828570485115, Accuracy: 97.34001922607422


In [16]:
#Run prototypical model with only training set
# This will be using the fourth values from the lists for the experiment
num_epochs = 20
num_episodes = 100
save_path = "./results/models/omniglot_train3.h5"

train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')


#number of classes
num_way = train_num_ways[3] 

#number of examples per class for support set
num_shot = num_shots[3]  

#number of query points
num_query = num_shots[3] 

least_loss = {'least_loss': 100.00}

support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

least_loss = {'least_loss': 100.00}

@tf.function
def train_step(support, query):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  train_acc.reset_states()

  for episode in range(num_episodes):
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    train_step(train_support, train_query)

  cur_loss = train_loss.result().numpy()
  if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100))

Saving new best model with loss:  2.6283963
Epoch 1, Loss: 2.6283962726593018, Accuracy: 38.27500534057617


Saving new best model with loss:  1.8341385
Epoch 2, Loss: 1.8341385126113892, Accuracy: 53.84999084472656


Saving new best model with loss:  1.4168556
Epoch 3, Loss: 1.4168555736541748, Accuracy: 61.57499694824219


Saving new best model with loss:  0.98337525
Epoch 4, Loss: 0.9833752512931824, Accuracy: 72.57500457763672


Saving new best model with loss:  0.7950602
Epoch 5, Loss: 0.7950602173805237, Accuracy: 78.05000305175781


...

Saving new best model with loss:  0.5219614
Epoch 9, Loss: 0.5219613909721375, Accuracy: 84.45000457763672


Saving new best model with loss:  0.50215197
Epoch 10, Loss: 0.5021519660949707, Accuracy: 85.44999694824219


Saving new best model with loss:  0.46327332
Epoch 11, Loss: 0.4632733166217804, Accuracy: 86.45001983642578
Epoch 12, Loss: 0.47207263112068176, Accuracy: 86.12501525878906


Saving new best model with loss:  0.42633688
Epoch 13, Loss: 0.4263368844985962, Accuracy: 87.27499389648438
Epoch 14, Loss: 0.4711262881755829, Accuracy: 87.02500915527344
Epoch 15, Loss: 0.42802906036376953, Accuracy: 87.70001220703125


Saving new best model with loss:  0.37522888
Epoch 16, Loss: 0.3752288818359375, Accuracy: 88.875
Epoch 17, Loss: 0.37922167778015137, Accuracy: 88.92500305175781
Epoch 18, Loss: 0.3955934941768646, Accuracy: 88.54999542236328
Epoch 19, Loss: 0.40708133578300476, Accuracy: 88.70000457763672
Epoch 20, Loss: 0.3811916410923004, Accuracy: 88.82498931884766


In [17]:
def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc
  

In [18]:
save_paths = ["./results/models/omniglot_train0.h5", "./results/models/omniglot_train1.h5", "./results/models/omniglot_train2.h5", "./results/models/omniglot_train3.h5"]

In [20]:
accuracies_proto_0 = {}
for save_path in save_paths:
  support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
  query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
  model = Prototypical(support, query, img_width, img_height, channels)
  model_path = save_path
  model.load(model_path)
  print("Model with path {} loaded.".format(save_path))
  
  num_episodes = 1000

  #number of classes
  num_way = test_num_ways[0]

  #number of examples per class for support set
  num_shot = num_shots[0]  

  #number of query points
  num_query = num_shots[0] 

  # Metrics to gather
  test_loss = tf.metrics.Mean(name='test_loss')
  test_acc = tf.metrics.Mean(name='test_accuracy')


  for i_episode in range(num_episodes):
    test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
    loss, acc = calc_loss(test_support, test_query)
    test_loss(loss)
    test_acc(acc)

  loss = test_loss.result().numpy()
  accuracy = test_acc.result().numpy() * 100
  print("Loss: ", loss)
  print("Accuracy: ", accuracy)
  accuracies_proto_0[model_path] = accuracy


Model with path ./results/models/omniglot_train0.h5 loaded.
Loss:  0.021341464
Accuracy:  99.41611289978027
Model with path ./results/models/omniglot_train1.h5 loaded.
Loss:  0.023788318
Accuracy:  99.40812587738037
Model with path ./results/models/omniglot_train2.h5 loaded.
Loss:  0.027576035
Accuracy:  99.1441547870636
Model with path ./results/models/omniglot_train3.h5 loaded.
Loss:  0.038555093
Accuracy:  99.07211661338806


In [21]:
accuracies_proto_1 = {}
for save_path in save_paths:
  model_path = save_path
  model.load(model_path)
  print("Model with path {} loaded.".format(save_path))
  num_episodes = 1000

  #number of classes
  num_way = test_num_ways[1]

  #number of examples per class for support set
  num_shot = num_shots[1]  

  #number of query points
  num_query = num_shots[1] 

  # Metrics to gather
  test_loss = tf.metrics.Mean(name='test_loss')
  test_acc = tf.metrics.Mean(name='test_accuracy')


  for i_episode in range(num_episodes):
    test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
    loss, acc = calc_loss(test_support, test_query)
    test_loss(loss)
    test_acc(acc)

  loss = test_loss.result().numpy()
  accuracy = test_acc.result().numpy() * 100
  print("Loss: ", loss)
  print("Accuracy: ", accuracy)
  accuracies_proto_1[model_path] = accuracy


Model with path ./results/models/omniglot_train0.h5 loaded.
Loss:  0.16401647
Accuracy:  96.19983434677124
Model with path ./results/models/omniglot_train1.h5 loaded.
Loss:  0.114926
Accuracy:  96.7598557472229
Model with path ./results/models/omniglot_train2.h5 loaded.
Loss:  0.20737296
Accuracy:  94.91981863975525
Model with path ./results/models/omniglot_train3.h5 loaded.
Loss:  0.12361249
Accuracy:  96.179860830307


In [22]:
accuracies_proto_2 = {}
for save_path in save_paths:
  model_path = save_path
  model.load(model_path)

  num_episodes = 1000

  #number of classes
  num_way = test_num_ways[2]

  #number of examples per class for support set
  num_shot = num_shots[2]  

  #number of query points
  num_query = num_shots[2] 

  # Metrics to gather
  test_loss = tf.metrics.Mean(name='test_loss')
  test_acc = tf.metrics.Mean(name='test_accuracy')

  print("Testing {} way {} shot".format(num_way, num_shot))
  print("Model with path {} loaded.".format(save_path))

  for i_episode in range(num_episodes):
    test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
    loss, acc = calc_loss(test_support, test_query)
    test_loss(loss)
    test_acc(acc)

  loss = test_loss.result().numpy()
  accuracy = test_acc.result().numpy() * 100
  print("Loss: ", loss)
  print("Accuracy: ", accuracy)
  accuracies_proto_2[model_path] = accuracy

Testing 20 way 5 shot
Model with path ./results/models/omniglot_train0.h5 loaded.
Loss:  0.06844688
Accuracy:  97.98349738121033
Testing 20 way 5 shot
Model with path ./results/models/omniglot_train1.h5 loaded.
Loss:  0.095079914
Accuracy:  97.65461087226868
Testing 20 way 5 shot
Model with path ./results/models/omniglot_train2.h5 loaded.
Loss:  0.09671981
Accuracy:  97.23567962646484
Testing 20 way 5 shot
Model with path ./results/models/omniglot_train3.h5 loaded.
Loss:  0.123219974
Accuracy:  97.12060689926147


In [23]:
accuracies_proto_3 = {}

def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

for save_path in save_paths:
  model_path = save_path
  model.load(model_path)
  print("Model with path {} loaded.".format(save_path))
  num_episodes = 1000

  #number of classes
  num_way = test_num_ways[3]

  #number of examples per class for support set
  num_shot = num_shots[3]  

  #number of query points
  num_query = num_shots[3] 

  # Metrics to gather
  test_loss = tf.metrics.Mean(name='test_loss')
  test_acc = tf.metrics.Mean(name='test_accuracy')


  for i_episode in range(num_episodes):
    test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
    loss, acc = calc_loss(test_support, test_query)
    test_loss(loss)
    test_acc(acc)

  loss = test_loss.result().numpy()
  accuracy = test_acc.result().numpy() * 100
  print("Loss: ", loss)
  print("Accuracy: ", accuracy)
  accuracies_proto_3[model_path] = accuracy

Model with path ./results/models/omniglot_train0.h5 loaded.
Loss:  0.40856043
Accuracy:  89.93017673492432
Model with path ./results/models/omniglot_train1.h5 loaded.
Loss:  0.30421925
Accuracy:  90.82520604133606
Model with path ./results/models/omniglot_train2.h5 loaded.
Loss:  0.58524114
Accuracy:  87.18504309654236
Model with path ./results/models/omniglot_train3.h5 loaded.
Loss:  0.37894273
Accuracy:  88.70013952255249


In [33]:
# Get the mean of all the accuracies per N way K shot experiment
accuracies_proto_0_avg = np.array(list(accuracies_proto_0.values())).mean()
accuracies_proto_1_avg = np.array(list(accuracies_proto_1.values())).mean()
accuracies_proto_2_avg = np.array(list(accuracies_proto_2.values())).mean()
accuracies_proto_3_avg = np.array(list(accuracies_proto_3.values())).mean()
print(accuracies_proto_0_avg)
print(accuracies_proto_1_avg)
print(accuracies_proto_2_avg)
print(accuracies_proto_3_avg)

99.26012754440308
96.0148423910141
97.49859869480133
89.1601413488388


In [24]:
#Combi Proto + Reptile for first values from the list ie 60-way 5-shot experiment

meta_step_size = 0.25

#Interval between running SGD on the validation dataset
eval_interval = 4

#number of classes
num_way = train_num_ways[0]

#number of examples per class for support set
num_shot = num_shots[0]  

#number of query points
num_query = num_shots[0] 


train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')
support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
query = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
model = Prototypical(support, query, img_height, img_width, channels)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate, beta_1=0)
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate)

num_epochs = 121
num_episodes = 100

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

@tf.function
def train_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_acc(acc)
  

@tf.function
def val_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  test_loss(loss)
  test_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  test_loss.reset_states()
  train_acc.reset_states()
  test_acc.reset_states()

  for episode in range(num_episodes):
    frac_done = episode / num_episodes
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    
    model = Prototypical(train_support, train_query, img_width, img_height, channels)
    model.call(train_support, train_query)
    old_weights = model.get_weights()
    train_step(train_support, train_query, optimizer_sgd)
    new_weights = model.get_weights()

    for part_weight in range(len(new_weights)):
        new_weights[part_weight] = old_weights[part_weight] + (
            (new_weights[part_weight] - old_weights[part_weight]) * cur_meta_step_size
        )

    model.set_weights(new_weights)
    if epoch % eval_interval == 0:
      eval_support, eval_query = get_next_episode(test_dataset, num_way, num_shot, num_query, no_test_classes)
      old_vars = model.get_weights()
      val_step(eval_support, eval_query, optimizer_adam)
      model.set_weights(old_vars)
      
  cur_loss = test_loss.result().numpy()

  if (epoch % 10 == 0):
       
    print("epoch ", epoch)

template = 'Epoch {}, Loss: {}, Accuracy: {}, ' \
                  'Test Loss: {}, Test Accuracy: {}'
print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100, test_loss.result(),
                            test_acc.result() * 100))

epoch  0
epoch  10
epoch  20
epoch  30
epoch  40
epoch  50
epoch  60
epoch  70
epoch  80
epoch  90
epoch  100
epoch  110
epoch  120
Epoch 121, Loss: 0.1321886032819748, Accuracy: 96.65668487548828, Test Loss: 0.06752282381057739, Test Accuracy: 97.72001647949219


In [25]:
#Combi Proto + Reptile for first values from the list ie 60-way 1-shot experiment

meta_step_size = 0.25

#Interval between running SGD on the validation dataset
eval_interval = 4

#number of classes
num_way = train_num_ways[1]

#number of examples per class for support set
num_shot = num_shots[1]  

#number of query points
num_query = num_shots[1] 


train_loss = tf.metrics.Mean(name='train_loss')
val_loss = tf.metrics.Mean(name='val_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
val_acc = tf.metrics.Mean(name='val_accuracy')
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')
support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
query = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
model = Prototypical(support, query, img_height, img_width, channels)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate, beta_1=0)
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate)

num_epochs = 121
num_episodes = 100
save_path = "./results/models/omniglot_train_reptile1.h5"

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

@tf.function
def train_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_acc(acc)
  

@tf.function
def val_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  test_loss(loss)
  test_acc(acc)


least_loss = {'least_loss': 100.00}
for epoch in range(num_epochs):
  train_loss.reset_states()
  test_loss.reset_states()
  train_acc.reset_states()
  test_acc.reset_states()

  for episode in range(num_episodes):
    frac_done = episode / num_episodes
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    
    model = Prototypical(train_support, train_query, img_width, img_height, channels)
    model.call(train_support, train_query)
    old_weights = model.get_weights()
    train_step(train_support, train_query, optimizer_sgd)
    new_weights = model.get_weights()

    for part_weight in range(len(new_weights)):
        new_weights[part_weight] = old_weights[part_weight] + (
            (new_weights[part_weight] - old_weights[part_weight]) * cur_meta_step_size
        )

    model.set_weights(new_weights)
    if epoch % eval_interval == 0:
      eval_support, eval_query = get_next_episode(test_dataset, num_way, num_shot, num_query, no_test_classes)
      old_vars = model.get_weights()
      val_step(eval_support, eval_query, optimizer_adam)
      model.set_weights(old_vars)
  if epoch % 10 ==0 :    
    print("Epoch", epoch + 1)
template = 'Epoch {}, Loss: {}, Accuracy: {}, ' \
                  'Test Loss: {}, Test Accuracy: {}'
print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100, test_loss.result(),
                            test_acc.result() * 100))

Epoch 1
Epoch 11
Epoch 21
Epoch 31
Epoch 41
Epoch 51
Epoch 61
Epoch 71
Epoch 81
Epoch 91
Epoch 101
Epoch 111
Epoch 121
Epoch 121, Loss: 0.4027485251426697, Accuracy: 89.40001678466797, Test Loss: 0.31943604350090027, Test Accuracy: 90.83334350585938


In [26]:
#Combi Proto + Reptile for first values from the list ie 40-way 5-shot experiment

meta_step_size = 0.25

#Interval between running SGD on the validation dataset
eval_interval = 4

#number of classes
num_way = train_num_ways[2]

#number of examples per class for support set
num_shot = num_shots[2]  

#number of query points
num_query = num_shots[2] 


train_loss = tf.metrics.Mean(name='train_loss')
val_loss = tf.metrics.Mean(name='val_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
val_acc = tf.metrics.Mean(name='val_accuracy')
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')
support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
query = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
model = Prototypical(support, query, img_height, img_width, channels)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate, beta_1=0)
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate)

num_epochs = 121
num_episodes = 100
save_path = "./results/models/omniglot_train_reptile2.h5"

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

@tf.function
def train_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_acc(acc)
  

@tf.function
def val_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  test_loss(loss)
  test_acc(acc)


least_loss = {'least_loss': 100.00}
for epoch in range(num_epochs):
  train_loss.reset_states()
  test_loss.reset_states()
  train_acc.reset_states()
  test_acc.reset_states()

  for episode in range(num_episodes):
    frac_done = episode / num_episodes
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    
    model = Prototypical(train_support, train_query, img_width, img_height, channels)
    model.call(train_support, train_query)
    old_weights = model.get_weights()
    train_step(train_support, train_query, optimizer_sgd)
    new_weights = model.get_weights()

    for part_weight in range(len(new_weights)):
        new_weights[part_weight] = old_weights[part_weight] + (
            (new_weights[part_weight] - old_weights[part_weight]) * cur_meta_step_size
        )

    model.set_weights(new_weights)
    if epoch % eval_interval == 0:
      eval_support, eval_query = get_next_episode(test_dataset, num_way, num_shot, num_query, no_test_classes)
      old_vars = model.get_weights()
      val_step(eval_support, eval_query, optimizer_adam)
      model.set_weights(old_vars)
  if epoch % 10 ==0 :    
    print("Epoch", epoch + 1)
template = 'Epoch {}, Loss: {}, Accuracy: {}, ' \
                  'Test Loss: {}, Test Accuracy: {}'
print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100, test_loss.result(),
                            test_acc.result() * 100))

Epoch 1
Epoch 11
Epoch 21
Epoch 31
Epoch 41
Epoch 51
Epoch 61
Epoch 71
Epoch 81
Epoch 91
Epoch 101
Epoch 111
Epoch 121
Epoch 121, Loss: 0.12245262414216995, Accuracy: 97.00999450683594, Test Loss: 0.0662667453289032, Test Accuracy: 98.11002349853516


In [27]:
#Combi Proto + Reptile for first values from the list ie 40-way 1-shot experiment

meta_step_size = 0.25

#Interval between running SGD on the validation dataset
eval_interval = 4

#number of classes
num_way = train_num_ways[3]

#number of examples per class for support set
num_shot = num_shots[3]  

#number of query points
num_query = num_shots[3] 


train_loss = tf.metrics.Mean(name='train_loss')
val_loss = tf.metrics.Mean(name='val_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
val_acc = tf.metrics.Mean(name='val_accuracy')
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')
support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
query = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
model = Prototypical(support, query, img_height, img_width, channels)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate, beta_1=0)
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate)

num_epochs = 121
num_episodes = 100

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

@tf.function
def train_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_acc(acc)
  

@tf.function
def val_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  test_loss(loss)
  test_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  test_loss.reset_states()
  train_acc.reset_states()
  test_acc.reset_states()

  for episode in range(num_episodes):
    frac_done = episode / num_episodes
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    
    model = Prototypical(train_support, train_query, img_width, img_height, channels)
    model.call(train_support, train_query)
    old_weights = model.get_weights()
    train_step(train_support, train_query, optimizer_sgd)
    new_weights = model.get_weights()

    for part_weight in range(len(new_weights)):
        new_weights[part_weight] = old_weights[part_weight] + (
            (new_weights[part_weight] - old_weights[part_weight]) * cur_meta_step_size
        )

    model.set_weights(new_weights)
    if epoch % eval_interval == 0:
      eval_support, eval_query = get_next_episode(test_dataset, num_way, num_shot, num_query, no_test_classes)
      old_vars = model.get_weights()
      val_step(eval_support, eval_query, optimizer_adam)
      model.set_weights(old_vars)
  if epoch % 10 ==0 :    
    print("Epoch", epoch + 1)
template = 'Epoch {}, Loss: {}, Accuracy: {}, ' \
                  'Test Loss: {}, Test Accuracy: {}'
print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100, test_loss.result(),
                            test_acc.result() * 100))

Epoch 1
Epoch 11
Epoch 21
Epoch 31
Epoch 41
Epoch 51
Epoch 61
Epoch 71
Epoch 81
Epoch 91
Epoch 101
Epoch 111
Epoch 121
Epoch 121, Loss: 0.33528900146484375, Accuracy: 90.92499542236328, Test Loss: 0.30321189761161804, Test Accuracy: 90.90000915527344


In [ ]:
def build_combined_dataset(train_dataset, val_dataset, test_dataset):
  total_rows = train_dataset.shape[0] + val_dataset.shape[0] + test_dataset.shape[0]
  train_dataset_images = int(train_dataset.shape[1]/2)
  test_dataset_images = int(train_dataset.shape[1]/4)
  val_dataset_images = int(train_dataset.shape[1]/4)
  combined_train_dataset = np.zeros([total_rows, train_dataset_images, img_height, img_width, channels], dtype=np.float32)
  combined_val_dataset = np.zeros([total_rows, val_dataset_images, img_height, img_width, channels], dtype=np.float32)
  combined_test_dataset = np.zeros([total_rows, test_dataset_images, img_height, img_width, channels], dtype=np.float32)

  for i in range(train_dataset.shape[0]):
    for j in range(train_dataset_images):
      combined_train_dataset[i][j] = train_dataset[i][j]
    for k in range(val_dataset_images):
      combined_val_dataset[i][k] = train_dataset[i][train_dataset_images + k]
      combined_test_dataset[i][k] = train_dataset[i][train_dataset_images + val_dataset_images + k]

  for i in range (val_dataset.shape[0]):
    for j in range(train_dataset_images):
      combined_train_dataset[train_dataset.shape[0] + i][j] = val_dataset[i][j]
    for k in range(val_dataset_images):
      combined_val_dataset[train_dataset.shape[0] + i][k] = val_dataset[i][train_dataset_images + k]
      combined_test_dataset[train_dataset.shape[0] + i][k] = train_dataset[i][train_dataset_images + val_dataset_images + k]

  for i in range (test_dataset.shape[0]):
    for j in range(train_dataset_images):
      combined_train_dataset[train_dataset.shape[0] + val_dataset.shape[0] + i][j] = test_dataset[i][j]
    for k in range(val_dataset_images):
      combined_val_dataset[train_dataset.shape[0] + val_dataset.shape[0] + i][k] = test_dataset[i][train_dataset_images + k]
      combined_test_dataset[train_dataset.shape[0] + val_dataset.shape[0] + i][k] = train_dataset[i][train_dataset_images + val_dataset_images + k]

  return combined_train_dataset, combined_val_dataset, combined_test_dataset

In [ ]:
combined_train_dataset, combined_val_dataset, combined_test_dataset = build_combined_dataset(train_dataset, val_dataset, test_dataset)
 

In [ ]:
print(combined_train_dataset.shape)
print(combined_val_dataset.shape)
print(combined_test_dataset.shape)

(6492, 10, 32, 32, 1)
(6492, 5, 32, 32, 1)
(6492, 5, 32, 32, 1)


In [ ]:
combined_train_dataset_reshaped = combined_train_dataset.reshape(64920, 32, 32, 1)
combined_val_dataset_reshaped = combined_val_dataset.reshape(32460, 32, 32, 1)
combined_test_dataset_reshaped = combined_test_dataset.reshape(32460, 32, 32, 1)

In [ ]:
train_labels = np.zeros(combined_train_dataset_reshaped.shape[0])
val_labels = np.zeros(combined_val_dataset_reshaped.shape[0])
test_labels = np.zeros(combined_test_dataset_reshaped.shape[0])
val = 0
for i in range(combined_train_dataset.shape[0]):
  for j in range(combined_train_dataset.shape[1]):
    train_labels[(i * 10) + j] = val
  val = val + 1

val = 0
for i in range(combined_val_dataset.shape[0]):
  for j in range(combined_val_dataset.shape[1]):
    val_labels[(i * 5) + j] = val
    test_labels[(i * 5) + j] = val
  val = val + 1



array([6450., 6450., 6450., 6450., 6450., 6450., 6450., 6450., 6450.,
       6450., 6451., 6451., 6451., 6451., 6451., 6451., 6451., 6451.,
       6451., 6451., 6452., 6452., 6452., 6452., 6452., 6452., 6452.,
       6452., 6452., 6452., 6453., 6453., 6453., 6453., 6453., 6453.,
       6453., 6453., 6453., 6453., 6454., 6454., 6454., 6454., 6454.,
       6454., 6454., 6454., 6454., 6454., 6455., 6455., 6455., 6455.,
       6455., 6455., 6455., 6455., 6455., 6455., 6456., 6456., 6456.,
       6456., 6456., 6456., 6456., 6456., 6456., 6456., 6457., 6457.,
       6457., 6457., 6457., 6457., 6457., 6457., 6457., 6457., 6458.,
       6458., 6458., 6458., 6458., 6458., 6458., 6458., 6458., 6458.,
       6459., 6459., 6459., 6459., 6459., 6459., 6459., 6459., 6459.,
       6459., 6460., 6460., 6460., 6460., 6460., 6460., 6460., 6460.,
       6460., 6460., 6461., 6461., 6461., 6461., 6461., 6461., 6461.,
       6461., 6461., 6461., 6462., 6462., 6462., 6462., 6462., 6462.,
       6462., 6462.,

In [ ]:
from numpy import argmax
from tensorflow.keras.utils import to_categorical
# one hot encode
train_labels_encoded = to_categorical(train_labels)
val_labels_encoded = to_categorical(val_labels)
test_labels_encoded = to_categorical(test_labels)
print(train_labels_encoded.shape)
print(val_labels_encoded.shape)
print(test_labels_encoded.shape)

(64920, 6492)
(32460, 6492)
(32460, 6492)


In [ ]:
from tensorflow.keras import applications
from tensorflow.keras.models import Sequential

base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,1))
x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(6492, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)
optimizer = tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer= optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(combined_train_dataset_reshaped, train_labels_encoded, epochs = 100, batch_size = 64, validation_data=(combined_val_dataset_reshaped, val_labels_encoded))


Epoch 1/100
1015/1015 [==============================] - 71s 63ms/step - loss: 10.2822 - accuracy: 7.7018e-05 - val_loss: 112.8816 - val_accuracy: 9.2421e-05
Epoch 2/100
1015/1015 [==============================] - 61s 60ms/step - loss: 9.6603 - accuracy: 1.8484e-04 - val_loss: 8.8060 - val_accuracy: 1.5404e-04
Epoch 3/100
1015/1015 [==============================] - 61s 60ms/step - loss: 8.9651 - accuracy: 3.8509e-04 - val_loss: 10.4517 - val_accuracy: 1.8484e-04
Epoch 4/100
1015/1015 [==============================] - 61s 60ms/step - loss: 8.6685 - accuracy: 3.0807e-04 - val_loss: 8.5928 - val_accuracy: 3.0807e-04
Epoch 5/100
1015/1015 [==============================] - 61s 60ms/step - loss: 8.5340 - accuracy: 3.2348e-04 - val_loss: 8.9433 - val_accuracy: 3.3888e-04
Epoch 6/100
1015/1015 [==============================] - 61s 61ms/step - loss: 8.4391 - accuracy: 3.0807e-04 - val_loss: 9.4202 - val_accuracy: 4.0049e-04
Epoch 7/100
1015/1015 [==============================] - 62s 61ms/

In [ ]:
preds = model.evaluate(combined_test_dataset_reshaped, test_labels_encoded)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

1015/1015 [==============================] - 14s 14ms/step - loss: 14.6651 - accuracy: 0.0147
Loss = 14.665081024169922
Test Accuracy = 0.014725816436111927
